In [29]:
import json
# Code from lecure to use json file credentials
with open('C:\\Users\\clint\\OneDrive\\Documents\\SQLPass.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']

import pyodbc

# Connecting jupyter to SQL. Code from the lecture.
conn = pyodbc.connect('Driver=SQL Server;'
                      'Server=DESKTOP-E8CUVPQ\SQLEXPRESS;'
                      'Database=DA320;'
                      'UID-' + username + ';'
                      'PWD' + password) 

In [ ]:
import urllib3

link = "https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected=2021&sort=desc&view=detailed"

# Construct a python request pool
urllib3.disable_warnings
http = urllib3.PoolManager(cert_reqs='CERT_NONE')

# Initiate a web request
r = http.request('GET', link, headers={'User-Agent': 'Mozilla/5.0'})

# Convert the result from raw bytes to text
datastring = str(r.data, "utf-8")

import re

# Search for titles using regular expression.
title = re.findall("class=\"title\"><h3>(.*)<\/h3><\/a>", datastring)

# Search for release date using regular expression.
releaseDate = re.findall("<div class=\"clamp-details\">\s+ <span>(.*)<\/span>", datastring)

# Search for metascore using regular expression.
score = re.findall("<div class=\"metascore_w large movie positive\">(.*)"
                   "<\/div>\n\s+<\/a>\n\s+<\/div>\n\s+<div class=\"clamp-userscore\">", datastring)

# Search for film summary using regular expression.
summary = re.findall("<div class=\"summary\">\n\s+(.*)", datastring)

# Search for thumbnail photo URL using regular expression.
image = re.findall("<img src=\"(.*)\" alt=\"(?:.*)\" \/><\/a>", datastring)

In [ ]:
#Code from the lecture to input webscraping from above into SQL table.
cursor = conn.cursor()
sql = "INSERT INTO MetaCritcFilms2020 (Title, ReleaseDate, Summary, MetaScore, Thumbnail) values(?,?,?,?,?)"
args = "Title", "ReleaseDate", "Sumary", "Metascore", "Thumbnail"
i = 0
while i < len(title) and i < len(releaseDate) and i < len(summary) and i < len(score) and i< len(image):
    args = title[i], releaseDate[i], summary[i], score[i], image[i]
    cursor.execute(sql, args)
    cursor.commit()
    i = i + 1

